In [ ]:
import os
import shutil

def remove_extracted_files(root_folder):
    """
    Remove all 'extracted_files' folders in the given root directory.
    """
    for folder_name in os.listdir(root_folder):
        project_folder = os.path.join(root_folder, folder_name)
        if os.path.isdir(project_folder):
            extracted_files_path = os.path.join(project_folder, 'extracted_files')
            if os.path.exists(extracted_files_path):
                shutil.rmtree(extracted_files_path)
                print(f"Removed: {extracted_files_path}")
            else:
                print(f"'extracted_files' not found in: {project_folder}")

if __name__ == "__main__":
    root_directory = os.getcwd()  # Use the current working directory
    remove_extracted_files(root_directory)


In [8]:
### File extraction from jar (app.jar and obfuscated_project.jar)###

import os
import zipfile


def find_class_from_jar(jar_path, class_name_with_package):
    """Find a class file (or its inner classes) in the given JAR."""
    with zipfile.ZipFile(jar_path, 'r') as jar:
        matching_files = [
            file for file in jar.namelist()
            if file.endswith(f"{class_name_with_package}.class") or file.startswith(f"{class_name_with_package}$")
        ]
        return matching_files  # Return all matching files
    return []


def extract_file_from_jar(jar_path, file_name, output_dir, relative_root):
    """Extract a file from a JAR and save it to the output directory."""
    with zipfile.ZipFile(jar_path, 'r') as jar:
        if file_name in jar.namelist():
            destination = os.path.join(output_dir, file_name.replace("/", os.sep))
            os.makedirs(os.path.dirname(destination), exist_ok=True)
            with open(destination, 'wb') as f:
                f.write(jar.read(file_name))
            relative_path = os.path.relpath(destination, relative_root)
            print(f"Extracted {file_name} to {relative_path}")
            return True
    return False


def process_project_folder(project_folder, relative_root):
    """Process a single project folder."""
    print(f"\nProcessing project folder: {os.path.relpath(project_folder, relative_root)}")

    # Define paths
    app_jar = os.path.join(project_folder, "app.jar")
    obfuscated_jar = os.path.join(project_folder, "obfuscated_project.jar")
    mapping_file = os.path.join(project_folder, "out.map")
    src_folder = project_folder  # Assuming .java files are directly inside

    if not (os.path.exists(app_jar) and os.path.exists(obfuscated_jar) and os.path.exists(mapping_file)):
        print(f"Required files (app.jar, obfuscated_project.jar, out.map) not found in {os.path.relpath(project_folder, relative_root)}. Skipping.")
        return

    # Create output directory for extracted files
    output_dir = os.path.join(project_folder, "extracted_files")
    os.makedirs(output_dir, exist_ok=True)

    for filename in os.listdir(src_folder):
        if filename.endswith(".java"):
            java_file_path = os.path.join(src_folder, filename)
            class_name = os.path.splitext(filename)[0]

            # Step 1: Find the fully qualified class name in the mapping file
            qualified_class_name, mapping_line = find_qualified_name(mapping_file, class_name)
            if not qualified_class_name:
                print(f"Qualified name for {class_name} not found in {os.path.relpath(mapping_file, relative_root)}")
                continue

            print(f"Mapping line for {class_name}: {mapping_line.strip()}")

            # Step 2: Find the corresponding class file(s) in app.jar
            class_files = find_class_from_jar(app_jar, qualified_class_name)
            if not class_files:
                print(f"Class files for {class_name} not found in {os.path.relpath(app_jar, relative_root)}")
                continue

            # Step 3: Extract the class files from app.jar
            for class_file in class_files:
                success = extract_file_from_jar(app_jar, class_file, output_dir, relative_root)
                if not success:
                    print(f"Failed to extract {class_file} from {os.path.relpath(app_jar, relative_root)}")

            # Step 4: Find the obfuscated class name(s) from the mapping file
            obfuscated_names = find_obfuscated_name(mapping_file, qualified_class_name)
            if not obfuscated_names:
                print(f"Obfuscated names for {class_name} not found in {os.path.relpath(mapping_file, relative_root)}")
                continue

            # Step 5: Extract the obfuscated class file(s) from obfuscated_project.jar
            for obfuscated_name in obfuscated_names:
                obfuscated_file_path = obfuscated_name + ".class"
                success = extract_file_from_jar(obfuscated_jar, obfuscated_file_path, output_dir, relative_root)
                if not success:
                    print(f"Obfuscated class file {obfuscated_file_path} not found in {os.path.relpath(obfuscated_jar, relative_root)}")


def find_qualified_name(mapping_file, original_class_name):
    """Find the fully qualified name for a class from the ProGuard mapping file."""
    with open(mapping_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '->' in line:
                # Format: original.ClassName -> obfuscated.ClassName:
                parts = line.split("->")
                if len(parts) == 2:
                    orig = parts[0].strip()
                    if orig.endswith(original_class_name):
                        return orig.replace('.', '/'), line
    return None, None


def find_obfuscated_name(mapping_file, qualified_class_name):
    """Find the obfuscated name(s) for a class (including inner classes) from the ProGuard mapping file."""
    obfuscated_names = []
    with open(mapping_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '->' in line:
                # Format: original.ClassName -> obfuscated.ClassName:
                parts = line.split("->")
                if len(parts) == 2:
                    orig = parts[0].strip()
                    obf = parts[1].strip().rstrip(':')
                    if orig == qualified_class_name.replace('/', '.'):
                        obfuscated_names.append(obf.replace('.', '/'))
    return obfuscated_names


def process_all_folders():
    """Process all project folders in the current working directory."""
    root_folder = os.getcwd()
    for item in os.listdir(root_folder):
        project_folder = os.path.join(root_folder, item)
        if os.path.isdir(project_folder):
            process_project_folder(project_folder, root_folder)


if __name__ == "__main__":
    # Process all project folders in the current directory
    process_all_folders()


Processing project folder: obfu
Mapping line for SingletonFactory: SingletonFactory -> a:
Extracted SingletonFactory.class to obfu\extracted_files\SingletonFactory.class
Extracted a.class to obfu\extracted_files\a.class

Processing project folder: obfu10
Mapping line for PicoAnnotationBuilder: org.picocontainer.PicoAnnotationBuilder -> a.b.b:
Extracted org/picocontainer/PicoAnnotationBuilder.class to obfu10\extracted_files\org\picocontainer\PicoAnnotationBuilder.class
Extracted a/b/b.class to obfu10\extracted_files\a\b\b.class

Processing project folder: obfu11
Mapping line for CDABuilder: org.slf4j.CDABuilder -> a.c.a:
Extracted org/slf4j/CDABuilder.class to obfu11\extracted_files\org\slf4j\CDABuilder.class
Extracted a/c/a.class to obfu11\extracted_files\a\c\a.class

Processing project folder: obfu12
Mapping line for ReviewBuilder: com.teleca.jamendo.api.ReviewBuilder -> a.a.a.a.b:
Extracted com/teleca/jamendo/api/ReviewBuilder.class to obfu12\extracted_files\com\teleca\jamendo\api\R

In [ ]:
# cleaning redundant files

import os
import shutil

def clean_obfu_folders(root_dir):
    """Remove specific subfolders and files from folders starting with 'obfu'."""
    for folder_name in os.listdir(root_dir):
        folder_path = os.path.join(root_dir, folder_name)
        
        # Process only folders starting with 'obfu'
        if os.path.isdir(folder_path) and folder_name.startswith('obfu'):
            print(f"Cleaning folder: {folder_name}")
            
            # Remove 'src' and 'bin' subfolders
            for subfolder in ['src', 'bin', '.settings']:
                subfolder_path = os.path.join(folder_path, subfolder)
                if os.path.exists(subfolder_path):
                    shutil.rmtree(subfolder_path)
                    print(f"Removed folder: {subfolder_path}")
            
            # Remove specific files
            for file_name in ['.project', '.classpath', ".progaurd_config.pro"]:
                file_path = os.path.join(folder_path, file_name)
                if os.path.exists(file_path):
                    os.remove(file_path)
                    print(f"Removed file: {file_path}")

# Specify the root directory
root_directory = os.getcwd()  # Current working directory

# Clean the folders
clean_obfu_folders(root_directory)


In [ ]:
# create empty folder namesd "decompiled_file"
import os

def create_obf_file_subfolder(root_dir):
    """
    Create a 'Obf_file' subfolder under every folder name that starts with 'obfu'.
    """
    for folder_name in os.listdir(root_dir):
        folder_path = os.path.join(root_dir, folder_name)

        # Check if the folder name starts with 'obf' and is a directory
        if os.path.isdir(folder_path) and folder_name.startswith("obfu"):
            obf_file_path = os.path.join(folder_path, "decompiled_file")
            
            # Create the 'Obf_file' subfolder if it doesn't already exist
            if not os.path.exists(obf_file_path):
                os.makedirs(obf_file_path)
                print(f"Created 'Obf_file' subfolder in: {folder_path}")
            else:
                print(f"'Obf_file' subfolder already exists in: {folder_path}")

# Specify the root directory
root_directory = os.getcwd()  # Current working directory

# Create 'Obf_file' subfolders
create_obf_file_subfolder(root_directory)


In [ ]:
# create empty folder namesd "src_file"
import os

def create_obf_file_subfolder(root_dir):
    """
    Create a 'Obf_file' subfolder under every folder name that starts with 'obfu'.
    """
    for folder_name in os.listdir(root_dir):
        folder_path = os.path.join(root_dir, folder_name)

        # Check if the folder name starts with 'obf' and is a directory
        if os.path.isdir(folder_path) and folder_name.startswith("obfu"):
            obf_file_path = os.path.join(folder_path, "src")
            
            # Create the 'Obf_file' subfolder if it doesn't already exist
            if not os.path.exists(obf_file_path):
                os.makedirs(obf_file_path)
                print(f"Created 'Obf_file' subfolder in: {folder_path}")
            else:
                print(f"'Obf_file' subfolder already exists in: {folder_path}")

# Specify the root directory
root_directory = os.getcwd()  # Current working directory

# Create 'Obf_file' subfolders
create_obf_file_subfolder(root_directory)